In [1]:
import cv2
import time
import numpy as np
import HandTrackingModule as htm
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

In [2]:
#wCam, hCam = 1280, 720
wCam, hCam = 640, 480
gestureThreshold = 600

detector = htm.handDetector()

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
volRange = volume.GetVolumeRange()
volume.SetMasterVolumeLevel(0, None)

minVol = volRange[0]
maxVol = volRange[1]

In [5]:
cap = cv2.VideoCapture(2)
cap.set(3, wCam)
cap.set(4, hCam)

pTime = 0
cTime = 0
delay = 30
mute_delay = 10
vol_selected_delay = 10

buttonPressed = False

plocX, plocY = 0, 0
clocX, clocY = 0, 0
frameR = 180
smooth = 1

mute_counter = 0
mode_counter = 0
option_counter = 0
vol_selected_counter = 0

mode = "ON"
option = "Volumen"
options = ["Volumen"]

vol = 0
volBar = 400
volPer = 0
smoothness = 5
muteVol = 0
vol_selected = 0

mute = "OF"

i = 0

while True :
    success, img = cap.read()
    img = detector.findHands(img)
    lmList, bbox = detector.findPosition(img, draw=False)

    if (len(lmList) != 0) :
        x1, y1 = lmList[0][1], lmList[0][2]
        x2, y2 = lmList[12][1], lmList[12][2]

        cx, cy = (x1+x2)//2, (y1+y2)//2

        fingers = detector.fingersUp(lmList)

        if mode == "ON": 
            if fingers == [0, 1, 1, 1, 1]:
                option_counter += 1
                if option_counter > delay:
                    option = options[i]
                    i+=1;
                    if i >= 3 :
                        i = 0

                    option_counter = 0

            if option == "Volumen" :
                cv2.line(img, (0, gestureThreshold), (wCam, gestureThreshold), (0, 255, 0), 10)
                length_mute, _ , info_mute = detector.findDistance(p1=4, p2=20, img=img, lmList=lmList, draw=False)
                length_selected_vol, _ , info_sel_vol = detector.findDistance(p1=4, p2=12, img=img, lmList=lmList, draw=False)

                if length_mute < 35 :
                    cv2.circle(img, (info_mute[4], info_mute[5]), 8, (0, 255, 0), cv2.FILLED)
                    mute_counter += 1
                    if mute_counter > mute_delay:
                        if muteVol == 0 :
                            volume.SetMute(1, None)
                            muteVol = 1
                            mute = "ON"
                        elif muteVol == 1 :
                            volume.SetMute(0, None)
                            muteVol = 0
                            mute = "OF"
                        mute_counter = 0
                    
                if length_selected_vol < 35 :
                    cv2.circle(img, (info_sel_vol[4], info_sel_vol[5]), 8, (0, 255, 0), cv2.FILLED)
                    vol_selected_counter += 1
                    if vol_selected_counter > vol_selected_delay:
                        if vol_selected == 0 :
                            vol_selected = 1
                        elif vol_selected == 1 :
                            vol_selected = 0
                        vol_selected_counter = 0

                if (muteVol!=1 and vol_selected!=1) :
                    length_volumen, _ , _ = detector.findDistance(p1=4, p2=8, img=img, lmList=lmList, draw=False)
                    volBar = np.interp(length_volumen, [5, 180], [400, 150])
                    volPer = np.interp(length_volumen, [5, 180], [0, 100])
                    volPer = smoothness * round(volPer/smoothness)

                    volume.SetMasterVolumeLevelScalar(volPer/100, None)

                cVol = int(volume.GetMasterVolumeLevelScalar() * 100)
                cv2.putText(img, f"Vol Set: {str(cVol)}", (20, 150), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)
                cv2.putText(img, f"MUTE: {mute}", (20, 200), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)        
        
            if fingers == [1, 1, 1, 1, 1] and cy <= gestureThreshold:
                mode_counter += 1
                if mode_counter > delay:
                    if mode == "ON" :
                        mode = "OF"
                    elif mode == "OF" :
                        mode = "ON"
                    mode_counter = 0

        elif cy <= gestureThreshold and mode == "OF" :
            if fingers == [1, 1, 1, 1, 1]:
                mode_counter += 1
                if mode_counter > delay:
                    if mode == "ON" :
                        mode = "OF"
                    elif mode == "OF" :
                        mode = "ON"
                    mode_counter = 0
    
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime

    cv2.putText(img, f"FPS: {str(int(fps))}", (20, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 255), 2)
    cv2.putText(img, f"MODE: {mode}", (20, 100), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 0, 0), 2)
    
    cv2.imshow("Image", img)
    cv2.waitKey(1)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        break